In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [5]:
data = pd.read_csv('spreadspoke_scores.csv')

In [6]:
data.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


In [7]:
data.shape

(13187, 17)

In [8]:
data.iloc[-1]

schedule_date                  12/27/2021
schedule_season                      2021
schedule_week                          16
schedule_playoff                    False
team_home              New Orleans Saints
score_home                            NaN
score_away                            NaN
team_away                  Miami Dolphins
team_favorite_id                      NaN
spread_favorite                       NaN
over_under_line                       NaN
stadium                 Caesars Superdome
stadium_neutral                     False
weather_temperature                  72.0
weather_wind_mph                      0.0
weather_humidity                      NaN
weather_detail                       DOME
Name: 13186, dtype: object

In [9]:
type(data['schedule_season'][0])

numpy.int64

In [10]:
current_season = data[data['schedule_season'] == 2021]
current_season.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
12947,9/9/2021,2021,1,False,Tampa Bay Buccaneers,31.0,29.0,Dallas Cowboys,TB,-7.5,51.5,Raymond James Stadium,False,NaN,NaN,NaN,NaN
12948,9/12/2021,2021,1,False,Atlanta Falcons,6.0,32.0,Philadelphia Eagles,ATL,-3.5,48,Mercedes-Benz Stadium,False,72.0,0.0,NaN,DOME
12949,9/12/2021,2021,1,False,Buffalo Bills,16.0,23.0,Pittsburgh Steelers,BUF,-6.5,48.5,Highmark Stadium,False,NaN,NaN,NaN,NaN
12950,9/12/2021,2021,1,False,Carolina Panthers,19.0,14.0,New York Jets,CAR,-5.0,44.5,Bank of America Stadium,False,NaN,NaN,NaN,NaN
12951,9/12/2021,2021,1,False,Cincinnati Bengals,27.0,24.0,Minnesota Vikings,MIN,-3.0,48,Paul Brown Stadium,False,NaN,NaN,NaN,NaN


In [11]:
current_season.shape

(240, 17)

In [12]:
def underdog_cover(team, team_id):
    team_df = current_season[(current_season['team_home'] == team) | (current_season['team_away'] == team)]
    team_df['actual_spread'] = abs(team_df['score_home'] - team_df['score_away'])
    team_mean_spread = team_df['actual_spread'].mean()
    team_underdogs = team_df[team_df['team_favorite_id'] != team_id]
    team_underdogs['spread_difference'] = team_underdogs['actual_spread'] - abs(team_underdogs['spread_favorite'])
    team_covered = team_underdogs[team_underdogs['spread_difference'] < 0]

    return team_mean_spread, team_covered
    


In [13]:
jags = current_season[(current_season['team_home'] == 'Jacksonville Jaguars') | (current_season['team_away'] == 'Jacksonville Jaguars')]
jags.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
12953,9/12/2021,2021,1,False,Houston Texans,37.0,21.0,Jacksonville Jaguars,JAX,-3.0,44.5,NRG Stadium,False,72.0,0.0,NaN,DOME
12970,9/19/2021,2021,2,False,Jacksonville Jaguars,13.0,23.0,Denver Broncos,DEN,-6.0,45.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN
12984,9/26/2021,2021,3,False,Jacksonville Jaguars,19.0,31.0,Arizona Cardinals,ARI,-8.0,51.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN
12995,9/30/2021,2021,4,False,Cincinnati Bengals,24.0,21.0,Jacksonville Jaguars,CIN,-7.5,46,Paul Brown Stadium,False,NaN,NaN,NaN,NaN
13018,10/10/2021,2021,5,False,Jacksonville Jaguars,19.0,37.0,Tennessee Titans,TEN,-4.5,49,TIAA Bank Field,False,NaN,NaN,NaN,NaN


In [14]:
jags['Actual Spread'] = abs(jags['score_home'] - jags['score_away'])
jags.head()

/var/folders/wr/nlq8phbn1_n186q38fbtdwcm0000gn/T/ipykernel_36989/3260473959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jags['Actual Spread'] = abs(jags['score_home'] - jags['score_away'])


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Actual Spread
12953,9/12/2021,2021,1,False,Houston Texans,37.0,21.0,Jacksonville Jaguars,JAX,-3.0,44.5,NRG Stadium,False,72.0,0.0,NaN,DOME,16.0
12970,9/19/2021,2021,2,False,Jacksonville Jaguars,13.0,23.0,Denver Broncos,DEN,-6.0,45.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,10.0
12984,9/26/2021,2021,3,False,Jacksonville Jaguars,19.0,31.0,Arizona Cardinals,ARI,-8.0,51.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,12.0
12995,9/30/2021,2021,4,False,Cincinnati Bengals,24.0,21.0,Jacksonville Jaguars,CIN,-7.5,46,Paul Brown Stadium,False,NaN,NaN,NaN,NaN,3.0
13018,10/10/2021,2021,5,False,Jacksonville Jaguars,19.0,37.0,Tennessee Titans,TEN,-4.5,49,TIAA Bank Field,False,NaN,NaN,NaN,NaN,18.0


In [15]:
jags_mean_spread = jags['Actual Spread'].mean()

In [16]:
jags_mean_spread

12.666666666666666

In [17]:
jags_underdogs = jags[jags['team_favorite_id'] != 'JAX']
jags_underdogs.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Actual Spread
12970,9/19/2021,2021,2,False,Jacksonville Jaguars,13.0,23.0,Denver Broncos,DEN,-6.0,45.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,10.0
12984,9/26/2021,2021,3,False,Jacksonville Jaguars,19.0,31.0,Arizona Cardinals,ARI,-8.0,51.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,12.0
12995,9/30/2021,2021,4,False,Cincinnati Bengals,24.0,21.0,Jacksonville Jaguars,CIN,-7.5,46,Paul Brown Stadium,False,NaN,NaN,NaN,NaN,3.0
13018,10/10/2021,2021,5,False,Jacksonville Jaguars,19.0,37.0,Tennessee Titans,TEN,-4.5,49,TIAA Bank Field,False,NaN,NaN,NaN,NaN,18.0
13035,10/17/2021,2021,6,False,Jacksonville Jaguars,23.0,20.0,Miami Dolphins,MIA,-1.5,47.5,Tottenham Hotspur Stadium,True,NaN,NaN,NaN,NaN,3.0


In [18]:
jags_underdogs['spread_difference'] = jags_underdogs['Actual Spread'] - abs(jags_underdogs['spread_favorite'])
jags_underdogs.head()

/var/folders/wr/nlq8phbn1_n186q38fbtdwcm0000gn/T/ipykernel_36989/2483087129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jags_underdogs['spread_difference'] = jags_underdogs['Actual Spread'] - abs(jags_underdogs['spread_favorite'])


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Actual Spread,spread_difference
12970,9/19/2021,2021,2,False,Jacksonville Jaguars,13.0,23.0,Denver Broncos,DEN,-6.0,45.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,10.0,4.0
12984,9/26/2021,2021,3,False,Jacksonville Jaguars,19.0,31.0,Arizona Cardinals,ARI,-8.0,51.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,12.0,4.0
12995,9/30/2021,2021,4,False,Cincinnati Bengals,24.0,21.0,Jacksonville Jaguars,CIN,-7.5,46,Paul Brown Stadium,False,NaN,NaN,NaN,NaN,3.0,-4.5
13018,10/10/2021,2021,5,False,Jacksonville Jaguars,19.0,37.0,Tennessee Titans,TEN,-4.5,49,TIAA Bank Field,False,NaN,NaN,NaN,NaN,18.0,13.5
13035,10/17/2021,2021,6,False,Jacksonville Jaguars,23.0,20.0,Miami Dolphins,MIA,-1.5,47.5,Tottenham Hotspur Stadium,True,NaN,NaN,NaN,NaN,3.0,1.5


In [19]:
jags_underdogs[jags_underdogs['spread_difference'] < 0]

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Actual Spread,spread_difference
12995,9/30/2021,2021,4,False,Cincinnati Bengals,24.0,21.0,Jacksonville Jaguars,CIN,-7.5,46,Paul Brown Stadium,False,NaN,NaN,NaN,NaN,3.0,-4.5
13074,11/7/2021,2021,9,False,Jacksonville Jaguars,9.0,6.0,Buffalo Bills,BUF,-14.5,48.5,TIAA Bank Field,False,NaN,NaN,NaN,NaN,3.0,-11.5
13088,11/14/2021,2021,10,False,Indianapolis Colts,23.0,17.0,Jacksonville Jaguars,IND,-10.5,48,Lucas Oil Stadium,False,72.0,0.0,NaN,DOME,6.0,-4.5


So there have only been 3 games this season where the Jags have been underdogs and covered a predicted spread

In [21]:
underdog_cover('Detroit Lions', 'DET')

/var/folders/wr/nlq8phbn1_n186q38fbtdwcm0000gn/T/ipykernel_36989/1076386230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['actual_spread'] = abs(team_df['score_home'] - team_df['score_away'])


(9.75,
       schedule_date  schedule_season schedule_week  schedule_playoff  \
 12983     9/26/2021             2021             3             False   
 13022    10/10/2021             2021             5             False   
 13046    10/24/2021             2021             7             False   
 13093    11/14/2021             2021            10             False   
 13101    11/21/2021             2021            11             False   
 13131     12/5/2021             2021            13             False   
 
                  team_home  score_home  score_away          team_away  \
 12983        Detroit Lions        17.0        19.0   Baltimore Ravens   
 13022    Minnesota Vikings        19.0        17.0      Detroit Lions   
 13046     Los Angeles Rams        28.0        19.0      Detroit Lions   
 13093  Pittsburgh Steelers        16.0        16.0      Detroit Lions   
 13101     Cleveland Browns        13.0        10.0      Detroit Lions   
 13131        Detroit Lions        2